In [1]:
import os

In [29]:
%pwd

'/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject'

In [11]:
%cd ..

/Users/yashlucky/Code/MLProjectsYoutube/chickenDiseaseClassification-mlproject


In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [93]:
data = pd.read_csv("artifacts/data_ingestion/train_data.csv")
data.head()

,images,label
0,salmo.1558.jpg,Salmonella
1,cocci.1866.jpg,Coccidiosis
2,cocci.171.jpg,Coccidiosis
3,salmo.1484.jpg,Salmonella
4,ncd.100.jpg,New Castle Disease


In [94]:
data['label'].unique()

array(['Salmonella', 'Coccidiosis', 'New Castle Disease', 'Healthy'],
      dtype=object)

In [102]:
from dataclasses import dataclass
from pathlib import Path

In [103]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [104]:
from cdclassifier.constants import *
from cdclassifier.utils.common import read_yaml, create_directories

In [105]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.param.IMAGE_SIZE,
            params_learning_rate = self.param.LEARNING_RATE,
            params_include_top = self.param.INCLUDE_TOP,
            params_weights = self.param.WEIGHTS,
            params_classes = self.param.CLASSES
        )
        return prepare_base_model_config


In [106]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
# from src.cdclassifier.utils.common import save_object

In [107]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config 
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[: -freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        # full_model.compile(
        #     optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
        #     loss=tf.keras.losses.CategoricalCrossentropy(),
        #     metrics=['accuracy']
        # )

        full_model.compile(
            # optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes= self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
        # model.export(path)
        # save_object(file_path=path, obj=model)

In [108]:
try: 
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e

[2024-08-19 05:19:40,867: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-19 05:19:40,870: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-19 05:19:40,871: INFO: common: created directory at: artifacts]
[2024-08-19 05:19:40,872: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-08-19 05:19:41,042: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-08-19 05:19:41,071: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.RMSprop` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.RMSprop`.]
[2024-08-19 05:19:41,072: WARNING: __init__: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.RMSprop`.]
Model: "model_